In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as tfl

2022-12-09 08:27:42.978324: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.metrics as metrics

In [3]:

x_train = np.load("./cleaned_data/x_train.npy")
y_train = np.load("./cleaned_data/y_train.npy")
train_mu = np.load("./cleaned_data/train_mu.npy")
train_std = np.load("./cleaned_data/train_std.npy")
x_val = np.load("./cleaned_data/x_val.npy")
y_val= np.load("./cleaned_data/y_val.npy")


In [4]:
# y = [no sack, sack, time to sack]
# x = (M, 23, 11)
print(x_train.shape)
print(x_val.shape)

(196648, 23, 11)
(42131, 23, 11)


In [5]:
import keras.backend as K

def my_loss(y_true, y_output):
    bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    mse = tf.keras.losses.MeanSquaredError()
    
    # get mse of only true positives
    true_sack_mask = y_true[:,1]==1

    if len(y_true[true_sack_mask]) == 0:
        return bce(y_true[:,0:-1], y_output[:,0:-1])
    else:
        return bce(y_true[:,0:-1], y_output[:,0:-1]) + mse(y_true[true_sack_mask][:,-1], y_output[true_sack_mask][:,-1])

def bce_metric(y_true, y_output):
    return K.mean(K.binary_crossentropy(y_true[:,0:-1], y_output[:,0:-1], from_logits=False))

def mse_metric(y_true, y_output):
    # get mse of only true positives
    true_sack_mask = y_true[:,1]==1
    if len(y_true[true_sack_mask]) == 0:
        return 0.0
    else:
        return K.mean(K.square(y_true[true_sack_mask][:,-1] - y_output[true_sack_mask][:,-1]), axis=-1)
    # return K.mean(K.square(y_pred[:,-1] - y_true[:,-1]), axis=-1)
    
def accuracy_metric(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    return K.mean(K.cast(y_true[:,1] == preds, 'float32'))

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * preds, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true[:,1], 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * preds, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(preds, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_loss(y_true, y_output):
    
    ground_positives = K.sum(y_true[:,1], axis=0) + K.epsilon()       # = TP + FN
    pred_positives = K.sum(y_output[:,0:-1], axis=0) + K.epsilon()         # = TP + FP
    true_positives = K.sum(y_true[:,0:-1] * y_output[:,0:-1], axis=0) + K.epsilon()  # = TP
        #all with shape (4,)
    
    precision = true_positives / pred_positives 
    recall = true_positives / ground_positives
        #both = 1 if ground_positives == 0 or pred_positives == 0
        #shape (4,)

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
        #still with shape (4,)

    weighted_f1 = f1 * ground_positives / K.sum(ground_positives) 
    weighted_f1 = K.sum(weighted_f1)

    
    return 1 - weighted_f1


In [6]:
''' 
Model

how cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md
'''

' \nModel\n\nhow cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md\n'

In [24]:
''' 
Output:
[prob of no sack, prob of sack, time till sack]

If predict no sack, doesn't matter what time is (recorded as -1 in training data)
'''

def createModel(input_shape = (23,7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    flat = tfl.Flatten(input_shape=(23, 7))(X)     # Flatten to pass into linear layers
    d1 = tfl.Dense(50, activation='relu')(flat)
    d3 = tfl.Dense(3,activation=None)(d1)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    intermediate = tfl.Reshape((3,1), input_shape=(3,))(d3)
    
    probs = tfl.Cropping1D(cropping=(0,1))(intermediate)
    probs = tfl.Reshape((2,), input_shape=(2,1))(probs)
    probs = tfl.Activation('softmax')(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(intermediate)
    time = tfl.Reshape((1,), input_shape=(1,1))(time)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=-1)([probs, time])
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model
    

In [25]:
model = createModel()

print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 23, 7)]      0           []                               
                                                                                                  
 flatten_2 (Flatten)            (None, 161)          0           ['input_3[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 50)           8100        ['flatten_2[0][0]']              
                                                                                                  
 dense_5 (Dense)                (None, 3)            153         ['dense_4[0][0]']                
                                                                                            

In [26]:
LEARNING_RATE = 0.000001
BETA_1 = 0.9
BETA_2 = 0.999
EPS = 1e-07

# Better optimizer
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=10000,
    decay_rate=0.9)

scheduled_opt = optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

In [27]:
opt = optimizers.Adam(
    learning_rate=LEARNING_RATE,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

model.compile(loss = my_loss, optimizer = scheduled_opt, metrics = [accuracy_metric, bce_metric, mse_metric, recall, precision])

In [29]:
NUM_EPOCHS = 3
history = model.fit(x_train[:,:,4:], y_train, epochs=NUM_EPOCHS, batch_size=128, validation_data=(x_val[:,:,4:], y_val))

Epoch 1/3
1537/1537 [==============================] - 4s 3ms/step - loss: 5.5997 - accuracy_metric: 0.2769 - bce_metric: 1.2736 - mse_metric: 4.3244 - recall: 0.7920 - precision: 0.0834 - val_loss: 2.1200 - val_accuracy_metric: 0.2877 - val_bce_metric: 1.2284 - val_mse_metric: 0.8883 - val_recall: 0.1892 - val_precision: 0.0763
Epoch 2/3
1537/1537 [==============================] - 4s 3ms/step - loss: 5.0889 - accuracy_metric: 0.3178 - bce_metric: 1.1714 - mse_metric: 3.9189 - recall: 0.7434 - precision: 0.0840 - val_loss: 1.9485 - val_accuracy_metric: 0.3323 - val_bce_metric: 1.1250 - val_mse_metric: 0.8203 - val_recall: 0.1755 - val_precision: 0.0756
Epoch 3/3
1537/1537 [==============================] - 4s 3ms/step - loss: 4.6955 - accuracy_metric: 0.3664 - bce_metric: 1.0687 - mse_metric: 3.6278 - recall: 0.6921 - precision: 0.0846 - val_loss: 1.7994 - val_accuracy_metric: 0.3848 - val_bce_metric: 1.0220 - val_mse_metric: 0.7745 - val_recall: 0.1607 - val_precision: 0.0762


In [30]:
train_no_sack = np.round(np.sum(y_train[:,0])/len(y_train), 3)
print(f"Percentage of training plays with no sack = {train_no_sack}")

Percentage of training plays with no sack = 0.919


In [21]:
model.predict(x_train[0:5,:,4:])

1/1 [==============================] - 0s 170ms/step


array([[0.9783481 , 0.02165189, 3.104981  ],
       [0.89377475, 0.10622519, 3.2740505 ],
       [0.688783  , 0.31121704, 0.719149  ],
       [0.95938665, 0.04061344, 1.6758628 ],
       [0.9656645 , 0.03433549, 1.8511863 ]], dtype=float32)

In [73]:
metrics_df = pd.DataFrame(history.history)


In [74]:
metrics_df.tail()

,loss,accuracy_metric,bce_metric,mse_metric,recall,precision,val_loss,val_accuracy_metric,val_bce_metric,val_mse_metric,val_recall,val_precision
0,-0.72843,0.514172,0.824179,5.20774,0.492617,0.082449,-0.822297,0.566703,0.739049,1.090825,0.110353,0.076884


In [31]:
num_no_sack = np.round(np.sum(y_val[:,0])/len(y_val), 3)
print(f"Percentage of validation plays with no sack = {num_no_sack}")

Percentage of validation plays with no sack = 0.924


In [32]:
val_loss, cat_acc, val_bce, val_mse, val_recall, val_precision = model.evaluate(x_val[:,:,4:], y_val, verbose=2)

1317/1317 - 2s - loss: 1.4525 - accuracy_metric: 0.3842 - bce_metric: 1.0228 - mse_metric: 0.4294 - recall: 0.0788 - precision: 0.0752 - 2s/epoch - 1ms/step


In [33]:

print(f"val loss = {val_loss}")
print(f"categorical accuracy = {cat_acc}")
print(f"val_bce = {val_bce}")
print(f"val_mse = {val_mse}")
print(f"val_recall = {val_recall}")
print(f"val_precision = {val_precision}")

val loss = 1.4524977207183838
categorical accuracy = 0.38421154022216797
val_bce = 1.022835373878479
val_mse = 0.42941030859947205
val_recall = 0.07880528271198273
val_precision = 0.07517759501934052


In [78]:
model.predict(x_val[:,:,4:])[:,0]

1317/1317 [==============================] - 3s 2ms/step


array([0.8433075 , 0.89811885, 0.89859354, ..., 0.47393072, 0.47523332,
       0.56478333], dtype=float32)

In [34]:
# https://www.kdnuggets.com/2021/02/saving-loading-models-tensorflow.html

model_string = f"models/fifth_model/weights_epochs{NUM_EPOCHS}"
model.save_weights(model_string)